In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
SIZE = 32

def add_ResidualLayer(inputs):
    x = layers.Conv2D(
        SIZE,
        kernel_size=(3, 3),
        padding='same',
        use_bias=False
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(
        SIZE,
        kernel_size=(3, 3),
        padding='same',
        use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, inputs])
    x = layers.Activation('relu')(x)
    
    return x


def add_ValueHead(model):
    model = layers.Conv2D(
        1,
        kernel_size=(1, 1),
        padding='same',
        use_bias=False
    )(model)
    model = layers.BatchNormalization()(model)
    model = layers.Activation('relu')(model)
    
    model = layers.Dense(SIZE, activation='relu')(model)
    model = layers.Dense(1, activation='tanh', name='value_out')(model)
    
    return model


def add_PolicyHead(model):
    model = layers.Conv2D(
        2,
        kernel_size=(1, 1),
        padding='same',
        use_bias=False
    )(model)
    model = layers.BatchNormalization()(model)
    model = layers.Activation('relu')(model)
    model = layers.Dense(8 * 8 + 1, name='policy_out')(model)
    
    return model


def get_model():
    inputs = layers.Input(shape=(8, 8, 2))
    model = layers.Conv2D(
        SIZE,
        kernel_size=(3, 3),
        padding='same',
        use_bias=False
    )(inputs)
    model = layers.BatchNormalization()(model)
    model = layers.Activation('relu')(model)
    
    for _ in range(10):
        model = add_ResidualLayer(model)
    
    value_out = add_ValueHead(model)
    policy_out = add_PolicyHead(model)
    
    ret = models.Model(
        inputs=inputs,
        outputs=[value_out, policy_out],
        name='kiwi-zero'
    )
    
    return ret

In [ ]:
FILE_NAME = '../2019-09-15 232553.txt'
X, y_values, y_policy = [], [], []

def encode_field(field):
    if field == 0:
        return [0, 0]
    elif field == -1:
        return [1, 0]
    else:
        return [0, 1]

with open(FILE_NAME) as file:
    for line in file:
        line = line.strip()
        line_split = line.split(' ')
        
        turn = int(line_split[-1])
        board = [encode_field(int(field) * turn) for field in line_split[:64]]
        policy = [float(prob) for prob in line_split[64:128 + 1]]
        value = float(line_split[-2])
        
        X.append(board)
        y_values.append(value)
        y_policy.append(policy)
        
X = np.array(X)
y_values = np.array(y_values)
y_policy = np.array(y_policy)

X = X.reshape(-1, 8, 8, 2)
print(X.shape)
print(y_values.shape)
print(y_policy.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_values_train, y_values_test, y_policy_train, y_policy_test = train_test_split(X, y_values, y_probs, test_size=0.2)

In [ ]:
losses = {
    "value_out": "mse",
    "policy_out": "mse",
}

loss_weights = {
    "value_out": 1.0,
    "policy_out": 1.0
}

model = get_model()
model.compile(
    optimizer=tf.train.AdamOptimizer(0.0003),
    loss=losses,
    loss_weights=loss_weights,
    metrics=['mae']
)

history = model.fit(
    X_train,
    { 'value_out': y_values_train, 'policy_out': y_policy_train },
    epochs=1000,
    batch_size=64,
    validation_split=0.2,
)